In [13]:
import pandas as pd
from PIL import Image
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tqdm import tqdm
from sklearn.model_selection import train_test_split

### Caricamento delle immagini in input in formato array numpy

In [5]:
# IMAGE_PATH_FOLDER = './Dati Sensori/Dataset Sensori Lavorato/1_M/1_M_1'
IMAGE_PATH_FOLDER = '../Dati Sensori/Dataset Distorto'

# Specificare solamente una tipologia di speckle
DISTORSIONI = ['gaussian blur nome prima parte cartella']
GRADI_DISTORSIONE = 5

HEIGHT = 512
WIDTH = 512

TARGET_SIZE = (WIDTH, HEIGHT)
CHANNELS = 1 # Scala di grigi

In [6]:
def get_sotto_cartelle(path):
    return [x[1] for x in os.walk(path)][0]

def get_immagini_cartella(path_cartella):
    immagini_cartella = []
    cartelle_soggetti = get_sotto_cartelle(path_cartella)
    for soggetto in tqdm(cartelle_soggetti):
        path_soggetto = f'{path_cartella}/{soggetto}'
        cartelle_acquisizioni = get_sotto_cartelle(path_soggetto)
        for acquisizione in cartelle_acquisizioni:
            path_acquisizione = f'{path_soggetto}/{acquisizione}'
            for filename in os.listdir(path_acquisizione):
                # if (filename.endswith(".jpg") or filename.endswith(".png"):
                if (filename.endswith("_1.jpg")):    
                    path_file = f'{path_acquisizione}/{filename}'
                    immagine = get_immagine_da_path(path_file)
                    immagini_cartella.append(immagine)
    return immagini_cartella
                    
def get_immagine_da_path(path_file):
    image = Image.open(path_file).convert('L')
    resized_image = image.resize(TARGET_SIZE)
    image_array = np.array(resized_image)
    normalized_image_array = image_array / 255.0
    return normalized_image_array


In [7]:
image_list = []
label_list = []

for distorsione in DISTORSIONI:
    path_distorsione = f'{IMAGE_PATH_FOLDER}/Dataset Sensori {distorsione}'
    for grado_distorsione in range(GRADI_DISTORSIONE):
        label = 100 - float(20 * (grado_distorsione))
        path_cartella_grado = f'{path_distorsione}{grado_distorsione}'
        immagini_grado = get_immagini_cartella(path_cartella_grado)
        image_list.extend(immagini_grado)
        label_to_append = [label for x in range(len(immagini_grado))] 
        label_list.extend(label_to_append)
        

100%|██████████| 14/14 [00:01<00:00, 10.82it/s]


In [8]:
train_images = np.array(image_list)
train_weights = np.array(label_list)

In [12]:
X = train_images
y = train_weights

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, CHANNELS)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  
])

In [16]:
model.compile(optimizer='adam', loss='mae', metrics = ['mae'])

In [28]:
model.fit(x=X_train, y=y_train, batch_size=1, epochs=30)

In [18]:
predictions = model.predict(X_test)

1/7 [===>..........................] - ETA: 0s

2023-07-22 17:13:13.467098: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 2s 315ms/step


In [19]:
predictions_list = [i[0] for i in predictions] 

In [26]:
max(abs(predictions_list  - y_test))

41.08346176147461

In [27]:
def avg(lst):
    return sum(lst) / len(lst)

max(abs(predictions_list - y_test))

41.08346176147461

In [30]:
model.save('../../Modelli/modello_pesi_inizializzati_gaussian_blur')

INFO:tensorflow:Assets written to: ../Modelli/modello_pesi_inizializzati_gaussian_blur/assets


INFO:tensorflow:Assets written to: ../Modelli/modello_pesi_inizializzati_gaussian_blur/assets
